##!중요 스프링 빈 조회 상속관계!

부모타입으로 조회하면 자식 타입도 함께 조회됨 즉 최고부모 Object 타입 조회시 모든 스프링빈 조회한다

BeanFactory 최상위 인터페이스  <-- ApplicationContext  Bean Factory의 기능을모두 상속받아 제공하는 인터페이스

BeanFactory <--> ApplicationContext 
Ac가 수많은 부가기능을 제공함
Ac의 부가기능 
MessageSource -> 한국에서 들어오면 한국어로 영어권에서는 영어로출력
EnvironmentCapable -> 로컬, 개발, 운영등을 구분해서 처리해줌
ApplicationEventPublisher -> 이벤트 발행및, 구독하는 모델을 편리하게지원
ResourceLoader -> 파일, 외부등에서 리소스를 편리하게 조회

BeanFactory OR AC를 스프링 컨테이너라고함

#스프링 빈 설정 메타 정보 BeanDefinition
BeanDefinition -> 빈설정 메타정보라고함
스프링컨테이너는 메타정보를 기반으로 스프링빈을 설정한다.

public class BeanDefinitionTest {

    //ApplicationContext로 getBean하면 안나옴
    AnnotationConfigApplicationContext ac = new AnnotationConfigApplicationContext(AppConfig.class);

    @Test
    @DisplayName("빈설정 메타정보 확인")
    void findApplicationBean() {
        String[] beanDefinitionNames = ac.getBeanDefinitionNames();
        for (String beanDefinitionName : beanDefinitionNames) {
            BeanDefinition beanDefinition = ac.getBeanDefinition(beanDefinitionName);

            if (beanDefinition.getRole() == BeanDefinition.ROLE_APPLICATION){
                System.out.println("beanDEFname = " + beanDefinitionName + " beanDEF = " + beanDefinition);

            }

        }

    }

}
실무에서 정의및 사용할일은 거의없다
-> 스프링 다양한형태의 설정정보를 BeanDef로 추상화해서 사용하는것이란걸 알면된다


#싱글톤 컨테이너
싱글톤 패턴 >객체가 자바안에 하나만 있는패턴

웹 app의 문제점-> 고객이 계속 요청할때마다 객체를 만들어야함


public class SingletonTEst {

    @Test
    @DisplayName("스프링 없는 순수한 DI 컨테이너")
    void pureContainer(){
        AppConfig appConfig = new AppConfig();
        //1 조회 호출 할때마다 객체생성
        MemberService memberService = appConfig.memberService();
        //2 조회 호출 할때마다 객체생성
        MemberService memberService1 = appConfig.memberService();

        System.out.println("memberservice1 = " + memberService);
        System.out.println("memberservice2 = " + memberService1);
//1번 memberservice1 = hello.core.member.MemberServiceImpl@2de23121
//2번 memberservice2 = hello.core.member.MemberServiceImpl@63475ace
        // 즉 요청이 계속와서 객체가 졸라쌓인다

        Assertions.assertThat(memberService).isNotSameAs(memberService1);
    }
}
순수 DI컨테이너 AppConfig는 요청이 초당 100개 들어오면 초당 객체가 100개 생성되고 소멸됨-> 메모리낭비가 너무크다
해결방안 -> 해당 객체가 딱 1개만 생성되고, 공유하도록 설계하면됨 ==> 싱글톤 패턴

#싱글톤 패턴
클래스 와 인스턴스가 딱 1개만 생성되는걸 보장하는 디자인 패턴
private 생성자를 사용 외부에서 생성되는걸 막아야함


public class SingletonService {

    private static final SingletonService instance = new SingletonService();
    //static 이라 되어있으면 static 영역에 하나만 만들어져 올라감
    public static SingletonService getInstance(){
        return instance;
    }

    private SingletonService(){
        
    }
    
    public void logic(){
        System.out.println("싱글톤 객체 로직 호출");
    }
    
}
getInstance() 메서드를 통해서만 조회가능 이 메서드를 호출하면 항상같은 인스턴스를 반환함
*생성자를 private로 막아서 혹시라도 외부에서 생성되는걸 막아야함

   @Test
    @DisplayName("싱글톤 패턴을 적용한 객체 사용")
    void singletonServiceTest(){
        SingletonService singletonservice1 = SingletonService.getInstance();
        SingletonService singletonservice2 = SingletonService.getInstance();

        System.out.println("1 + " + singletonservice1);
        System.out.println("2 + " + singletonservice2);

    }
    //1 + hello.core.singleton.SingletonService@c0c2f8d
    //2 + hello.core.singleton.SingletonService@c0c2f8d
}
같은 객체가 반환

싱글톤 코드의 문제
1.코드 구현하는 코드자체가 많이들어감
2.DIP를 위반(클라이언트가 구체클래스에 의존해야함)
3.OCP위반할가능성이높고 테스트하기 힘듬==> 유연성이 떨어짐
4.private 생성자를 써서 자식클래스 만들기 어려움


#==>> 스프링컨테이너를 쓰면 객체를 다 싱글톤으로 만들어서 관리해줌 문제점도 전부다 해결하고 장점만 가져감

##싱글톤 컨테이너
스프링컨테이너는 싱글톤 패턴 적용안해도 객체인스턴스를 싱글톤으로 관리해줌 (요청이 올때마다 이미만들어진 객체를 사용해 효율적으로 사용가능하다)

#싱글톤 방식의 주의점
여러 클라이언트가 하나의 같은 객체 인스턴스를 공유하기때문에 싱글톤객체는 상태를 유지stateful하게 설계하면 안됨
(싱글톤)스프링 필드에 공유값을 설정하면 큰 장애가 발생함
스프링빈은 항상 무상태로 설계해야함

@Configuration 에서도 여러번 호출될거같지만 결국 싱글톤 보장해줌
자바코드상에선 메소드가 memberRepository가 3번 호출되어야하는게 맞는데 한번만 호출됨




